In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from matplotlib.ticker import ScalarFormatter, FuncFormatter
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from statsmodels.tsa.arima.model import ARIMA
from ipywidgets import interact
import matplotlib.ticker as ticker
import warnings

# Suprimir advertencias
warnings.filterwarnings('ignore')

df_precio_real = pd.read_excel("Precio Historico Carbon.xlsx", sheet_name = 'Historico')

# Convertir la columna 'Fecha' a formato datetime
df_precio_real['Fecha'] = pd.to_datetime(df_precio_real['Fecha'], format='%d/%m/%Y')

# Convertir todos los nombres de pilas a minúsculas
df_precio_real['Pila'] = df_precio_real['Pila'].str.lower()

# Filtrar los registros donde la columna 'Pila' sea igual a 'Pila 4SD'
df_precio_real = df_precio_real[df_precio_real['Pila'] == 'pila 4sd']

# Promediar los precios por día
df_precio_real = df_precio_real.groupby(['Fecha']).agg({'Precio suministro': 'mean'}).reset_index()

# Crear una lista de años únicos
anios = df_precio_real['Fecha'].dt.year.unique()
anios = sorted(anios)

# Crear una lista de trimestres
trimestres = ['TODOS','Q1', 'Q2', 'Q3', 'Q4']

# Crear un widget de selección de año
anio_selector = widgets.Dropdown(
    options=anios,
    value=anios[-1],
    description='Año:',
    disabled=False,
)

trimestre_selector = widgets.Dropdown(
    options=trimestres,
    value='TODOS',
    description='Trimestre:',
    disabled=False,
)

# Función para actualizar la gráfica
def actualizar_grafica(anio, trimestre):
    # Filtrar por año específico
    df_filtrado = df_precio_real[df_precio_real['Fecha'].dt.year == anio]
    
    # Filtrar por trimestre específico
    if trimestre == 'Q1':
        df_filtrado = df_filtrado[(df_filtrado['Fecha'].dt.month >= 1) & (df_filtrado['Fecha'].dt.month <= 3)]
    elif trimestre == 'Q2':
        df_filtrado = df_filtrado[(df_filtrado['Fecha'].dt.month >= 4) & (df_filtrado['Fecha'].dt.month <= 6)]
    elif trimestre == 'Q3':
        df_filtrado = df_filtrado[(df_filtrado['Fecha'].dt.month >= 7) & (df_filtrado['Fecha'].dt.month <= 9)]
    elif trimestre == 'Q4':
        df_filtrado = df_filtrado[(df_filtrado['Fecha'].dt.month >= 10) & (df_filtrado['Fecha'].dt.month <= 12)]
    elif trimestre == 'TODOS':
        df_filtrado = df_filtrado[(df_filtrado['Fecha'].dt.month >= 1) & (df_filtrado['Fecha'].dt.month <= 12)]

    # Asegurarse de que los datos estén ordenados cronológicamente por fecha
    df_filtrado = df_filtrado.sort_values(by='Fecha')

    techo = 400000
    piso = 200000

    # Crear la gráfica
    plt.figure(figsize=(14, 8))

    # Plotear las series originales
    plt.plot(df_filtrado['Fecha'], df_filtrado['Precio suministro'], label='Precio Suministro', linewidth=2, color='green', alpha=0.5)
    
    # Añadir líneas punteadas para techo y piso
    plt.axhline(y=techo, color='brown', linestyle='--', linewidth=1, label='Techo')
    plt.axhline(y=piso, color='brown', linestyle='--', linewidth=1, label='Piso')
    
    plt.title(f'Comparación de Precios Año {anio} - {trimestre}')
    plt.xlabel('Fecha')
    plt.ylabel('Precio')
    plt.gca().yaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=False))
    plt.ticklabel_format(style='plain', axis='y', useOffset=False)
    plt.legend()
    plt.grid(True)
    
    # Ajustar los límites del eje y para mostrar ambas series correctamente
    min_y = min(200000, df_filtrado['Precio suministro'].min()) * 0.95
    max_y = max(400000, df_filtrado['Precio suministro'].max()) * 1.05
    plt.ylim(min_y - 10, max_y + 10)
    
    plt.show()

# Mostrar los selectores de año y trimestre y la gráfica
display(anio_selector, trimestre_selector)
widgets.interactive_output(actualizar_grafica, {'anio': anio_selector, 'trimestre': trimestre_selector})

Dropdown(description='Año:', index=4, options=(2020, 2021, 2022, 2023, 2024), value=2024)

Dropdown(description='Trimestre:', options=('TODOS', 'Q1', 'Q2', 'Q3', 'Q4'), value='TODOS')

Output()